# Shor's Algorithm
This implementation was authored by Gregory Croisdale <br>
5/1/2020, COSC 494, University of Tennesse, Knoxville

In [15]:
import numpy as np
import matplotlib.pyplot as plt

In [68]:
def collapse(n, a, rem):
    """Find states which yield certain remainder"""
    end = 2 ** n.bit_length()
    out = []
    for i in range(end):
        if (a ** i) % n == rem:
            out += [i]
    return out

In [71]:
def quantum_ft(n, guess, rem=3):
    """Performs a quantum fourier transformation so we can find periods. Large
       portions of this code are from the Qiskit reference material.           """
    from qiskit import QuantumCircuit, QuantumRegister, execute, Aer, IBMQ
    from numpy import pi
    
    # The following code is from qiskit reference:
    def qft_rotations(circuit, n):
        """Performs qft on the first n qubits in circuit (without swaps)"""
        if n == 0:
            return circuit
        n -= 1
        circuit.h(n)
        for qubit in range(n):
            circuit.cu1(pi/2**(n-qubit), qubit, n)
        # At the end of our function, we call the same function again on
        # the next qubits (we reduced n by one earlier in the function)
        qft_rotations(circuit, n)
    
    def swap_registers(circuit, n):
        for qubit in range(n//2):
            circuit.swap(qubit, n-qubit-1)
        return circuit

    def qft(circuit, n):
        """QFT on the first n qubits in circuit"""
        qft_rotations(circuit, n)
        swap_registers(circuit, n)
        return circuit
    
    # The following code is my own:
    # mark qubits with designated modulo value
    marked = collapse(n, guess, rem)
    print(marked)
    
    # create a democratic state from all marked qubits
    state = [1 / np.sqrt(len(marked)) if i in marked else 0 for i in range(2 ** n.bit_length())]
    q = QuantumRegister(n.bit_length())
    qc = QuantumCircuit(q)
    qc.initialize(state, q)
    qft(qc,n.bit_length())
    
    backend = Aer.get_backend("statevector_simulator")
    statevector = execute(qc, backend=backend).result().get_statevector()
    return statevector

[3, 13, 23, 33, 43, 53, 63]


array([ 3.30718914e-01-1.44647807e-16j,  2.89535777e-02+2.85167850e-03j,
        3.09618940e-02+6.15870363e-03j,  3.51178176e-02+1.06528735e-02j,
        4.36492051e-02+1.80800927e-02j,  6.53635953e-02+3.49375696e-02j,
        1.97490218e-01+1.31958745e-01j, -1.10956324e-01-9.10595022e-02j,
       -3.34076552e-02-3.34076552e-02j, -1.47645862e-02-1.79907004e-02j,
       -5.22109683e-03-7.81392361e-03j,  2.82399697e-03+5.28332672e-03j,
        1.80800927e-02+4.36492051e-02j, -8.87649789e-02-2.92618920e-01j,
       -1.37944418e-02-6.93493421e-02j, -5.02479175e-03-5.10175668e-02j,
       -4.33943421e-17-4.72455591e-02j,  5.02479175e-03-5.10175668e-02j,
        1.37944418e-02-6.93493421e-02j,  8.87649789e-02-2.92618920e-01j,
       -1.80800927e-02+4.36492051e-02j, -2.82399697e-03+5.28332672e-03j,
        5.22109683e-03-7.81392361e-03j,  1.47645862e-02-1.79907004e-02j,
        3.34076552e-02-3.34076552e-02j,  1.10956324e-01-9.10595022e-02j,
       -1.97490218e-01+1.31958745e-01j, -6.53635953

In [11]:
def classical_ft(n):
    """Performs a classical fourier transformation so we can find periods."""
    return []

In [12]:
def find_period(ft_result):
    """Finds the period from a completed fourier transformation."""
    return []

In [75]:
def Shor_Factor(
    n,
    guess=3,
    mode="quantum"):
    """Factors an integer using a designated method"""
    if (mode == "quantum"):
        return find_period(quantum_ft())
    elif (mode == "classical"):
        return find_period(quantum_ft())
    else:
        raise TypeError(mode)
    return n

In [78]:
Shor_Factor(2, mode="pee")

TypeError: pee